In [1]:
import calendar
import datetime
import json
import pathlib
from typing import Optional

import jellyfish
import numpy as np
import pandas as pd

pd.set_option('display.max_columns', 130)

In [2]:
TOP_DIR = pathlib.Path.cwd().parent

CAUSES = {
    'AC': 'accident',
    'S': 'suicide',
    'H': 'homicide',
    'M': 'illness',
    'DA': 'acute drugs/alcohol problems',
    'O': 'other',
}

FMT = '%m/%d/%Y'
OUTPUT_FMT = '%Y-%m-%d'

MONTHS = dict(zip(['January', 'February', 'March', 'April', 'May', 'June', 'July',
                   'August', 'September', 'October', 'November', 'December'], range(1,13)))

# load nested structure and flatten
with open(TOP_DIR / 'code_lookup' / 'cntyfips_to_many.json') as f:
    FIPS_DICT = json.load(f)

FIPS = []
for fip, vals in FIPS_DICT.items():
    for val in vals:
        d = val
        d['fips'] = fip
        FIPS.append(d)

In [3]:
def create_date_range(row: pd.Series) -> dict[datetime.datetime]:
    """Create the start and end date for the incident, starting with most specific to least"""
    year, dod_str = row

    if pd.isnull(dod_str):  # full year
        start = datetime.datetime(year, 1, 1)
        end = datetime.datetime(year, 12, 31)
    elif '/' in dod_str:  # exact date
        month, day, year = [int(c) if c != 'xx' else None for c in dod_str.strip().split('/')]

        if day is None:  # month range
            start = datetime.datetime(year, month, 1)
            last_day = calendar.monthrange(year, month)[-1]
            end = datetime.datetime(year, month, last_day)
        else:
            start = datetime.datetime(year, month, day)
            end = start + datetime.timedelta(days=1)
    else:  # month range
        month = MONTHS.get(dod_str.strip())
        start = datetime.datetime(year, month, 1)
        last_day = calendar.monthrange(year, month)[-1]
        end = datetime.datetime(year, month, last_day)

    return {'start': start, 'end': end}

In [4]:
def split_date(dtstr: Optional[str]) -> dict[int]:
    if pd.isnull(dtstr):
        return {}

    s = dtstr.strip()

    if '/' in dtstr:
        month, day, year = [int(c) if c != 'xx' else None for c in s.split('/')]
    else:
        day, year = None, None
        month = MONTHS.get(s)

    return {'year': year, 'month': month, 'day': day}

def compare_names(row: pd.Series) -> float:
    a, b = row
    return jellyfish.jaro_distance(a.lower(), b.lower())

In [5]:
jails = pd.read_csv('all_jails.csv', encoding='WINDOWS-1252', dtype={'fips': pd.Int64Dtype()})
jails = jails.dropna(subset='fips')
jails['fips'] = jails['fips'].apply(lambda i: f'0{i}' if i <= 9999 else str(i))  # convert to 5-char string

deaths = pd.read_csv('all_deaths.csv', encoding='WINDOWS-1252')
deaths['cause_desc'] = deaths['cause_short'].apply(lambda s: CAUSES.get(s))
deaths['suicides'] = np.where(deaths['cause_short'] == 'S', 1, 0)
deaths['deaths'] = 1  # since each row represents a death

deaths['date_range'] = deaths[['year', 'date_of_death']].apply(create_date_range, axis=1)
deaths['start_date'] = deaths['date_range'].apply(lambda d: d.get('start'))
deaths['end_date'] = deaths['date_range'].apply(lambda d: d.get('end'))

# deaths['dod_split'] = deaths['date_of_death'].apply(split_date)
# deaths['dod_year'] = deaths['dod_split'].apply(lambda d: d.get('year')).astype(pd.Int64Dtype())
# deaths['dod_year'] = np.where(deaths['dod_year'].notnull(), deaths['dod_year'], deaths['year'])
# deaths['dod_month'] = deaths['dod_split'].apply(lambda d: d.get('month')).astype(pd.Int64Dtype())
# deaths['dod_day'] = deaths['dod_split'].apply(lambda d: d.get('day')).astype(pd.Int64Dtype())

# merge
jail_cols = ['id', 'fips']
death_jails = pd.merge(jails[jail_cols], deaths, on='id').rename(columns={'id': 'reuters_id'})

# create or load mapping file
mapping_fname = pathlib.Path('mapping.csv')

if not mapping_fname.exists():
    fips = pd.DataFrame(FIPS)

    cols = ['id', 'fips', 'state', 'jail', 'county', 'FACID', 'COUNTY', 'FACNAME']
    mapping = pd.merge(jails, fips, left_on=['fips'], right_on=['fips'])[cols]
    mapping['score'] = mapping[['jail', 'FACNAME']].apply(compare_names, axis=1)
    mapping['rank'] = mapping.groupby(['fips', 'jail'])['score'].rank(method='dense', ascending=False)
    mapping.sort_values(['fips', 'jail', 'rank'])

    # top matched results
    mapping = mapping.loc[mapping['rank'] == 1, :]
    assert (mapping.duplicated(keep=False).sum() == 0)
    mapping = mapping[['id', 'FACID']].rename(columns={'id': 'reuters_id', 'FACID': 'id'})

    mapping.to_csv(mapping_fname, index=False)
else:
    mapping = pd.read_csv(mapping_fname)

In [6]:
df = pd.merge(death_jails, mapping, on='reuters_id')

df['source_url'] = 'https://www.reuters.com/investigates/special-report/usa-jails-graphic/'
df['source_url_2'] = 'https://graphics.thomsonreuters.com/data/jails/Allstatesinsurvey.zip'

export_cols = ['id', 'start_date', 'end_date', 'deaths', 'suicides', 'source_url', 'source_url_2']

df = (df.groupby(['id', 'start_date', 'end_date', 'source_url', 'source_url_2'])
        .agg({'suicides': np.sum, 'deaths': np.sum})).reset_index()

(df['end_date'] - df['start_date']).value_counts()

df[export_cols].to_csv('incidents.csv', index=False)